In [1]:
# https://pygis.io/docs/e_raster_rasterize.html
# https://rasterio.readthedocs.io/en/stable/topics/features.html

# Use this script to convert a vector polygon file of TAZ accessibility to a raster file that new PPA tool can read
# for computing driving accessibility.

from pathlib import Path

import pandas as pd
import arcpy
from shapely import wkt
import numpy as np
import geopandas as gpd
import rasterio
from rasterio import features

def esri_to_df(esri_obj_path, include_geom, field_list=None, index_field=None, 
               crs_val=None, dissolve=False):
    """
    Converts ESRI file (File GDB table, SHP, or feature class) to either pandas dataframe
    or geopandas geodataframe (if it is spatial data)
    esri_obj_path = path to ESRI file
    include_geom = True/False on whether you want resulting table to have spatial data (only possible if
        input file is spatial)
    field_list = list of fields you want to load. No need to specify geometry field name as will be added automatically
        if you select include_geom. Optional. By default all fields load.
    index_field = if you want to choose a pre-existing field for the dataframe index. Optional.
    crs_val = crs, in geopandas CRS string format, that you want to apply to the resulting geodataframe. Optional.
    dissolve = True/False indicating if you want the resulting GDF to be dissolved to single feature.
    """

    fields = field_list # should not be necessary, but was having issues where class properties were getting changed with this formula
    if not field_list:
        fields = [f.name for f in arcpy.ListFields(esri_obj_path)]

    if include_geom:
        import geopandas as gpd
        # by convention, geopandas uses 'geometry' instead of 'SHAPE@' for geom field
        f_esrishp = 'SHAPE@'
        f_gpdshape = 'geometry'
        fields = fields + [f_esrishp]

    data_rows = []
    with arcpy.da.SearchCursor(esri_obj_path, fields) as cur:
        for row in cur:
            rowlist = [i for i in row]
            if include_geom:
                geom_wkt = wkt.loads(rowlist[fields.index(f_esrishp)].WKT)
                rowlist[fields.index(f_esrishp)] = geom_wkt
            out_row = rowlist
            data_rows.append(out_row)  

    if include_geom:
        fields_gpd = [f for f in fields]
        fields_gpd[fields_gpd.index(f_esrishp)] = f_gpdshape
        
        out_df = gpd.GeoDataFrame(data_rows, columns=fields_gpd, geometry=f_gpdshape)

        # only set if the input file has no CRS--this is not same thing as .to_crs(), which merely projects to a CRS
        if crs_val: out_df.crs = crs_val

        # dissolve to single zone so that, during spatial join, points don't erroneously tag to 2 overlapping zones.
        if dissolve and out_df.shape[0] > 1: 
            out_df = out_df.dissolve() 
    else:
        out_df = pd.DataFrame(data_rows, index=index_field, columns=field_list)

    return out_df


In [35]:

# vector_fc is a TAZ polygon file. For each polygon it has the accessibility value.
vector_fc = r"I:\Projects\Darren\PPA3_GIS\PPA3_GIS.gdb\TEST_taz_drive_job_acc" # vector you want to convert to raster
ref_tif = r"\\Arcserverppa-svr\PPA_SVR\PPA_03_01\PPA3_GIS_SVR\access_tif\pop2020.tif" # use this tif's metadata to build fitting raster from vector

val_field = 'EMPdcurv'

tifdata = rasterio.open(ref_tif)
epsg_id = tifdata.crs.to_epsg()
starting_vector_crs = 'EPSG:2226'
tif_crs = f"EPSG:{epsg_id}"
vector_gdf = esri_to_df(esri_obj_path=vector_fc, include_geom=True, field_list=[val_field],
               crs_val=starting_vector_crs)

vector_gdf = vector_gdf.to_crs(tif_crs) # tif and vector must have same CRS

shapes_values = ((geom, val) for geom, val in zip(vector_gdf.geometry, vector_gdf[val_field]))


# Rasterize vector using the shape and transform of the raster
# https://rasterio.readthedocs.io/en/stable/api/rasterio.features.html#rasterio.features.rasterize
rasterized = features.rasterize(shapes_values,
                                out_shape = tifdata.shape,
                                transform = tifdata.transform,
                                all_touched = True,
                                default_value=0,
                                fill = 0,   # background value where there's no data
                                dtype = float)

# output raster to new tif
output_tif = Path(ref_tif).parent.joinpath(f"{val_field}.tif")

with rasterio.open(output_tif, "w",
        driver = "GTiff",
        crs = tifdata.crs,
        transform = tifdata.transform,
        dtype = rasterio.float32,
        count = 1,
        width = tifdata.width,
        height = tifdata.height) as dst:
    dst.write(rasterized, indexes = 1)

print(rasterized.max())
print(f"success. output TIF in {output_tif}")

399403.0
success. output TIF in \\Arcserverppa-svr\PPA_SVR\PPA_03_01\PPA3_GIS_SVR\access_tif\EMPdcurv.tif
